# Flux Balance and Flux Variability Analysis
# Adopted from this tutorial in CobraPpy
# source (https://cobrapy.readthedocs.io/en/latest/simulating.html#Running-FVA)

In [ ]:
# reading the input files of the iMAT models
input_path = '/path/to/iMAT_output'
FBA_output = '/path/to/FBAoutput/folder/'
FVA_output = '/path/to/FVAoutput/folder/'

In [2]:
# note: this is running on a python virtual environment of version 3.9.6 set on terminal and activated via conda to be used as a kernel here 
# import the necessary packages 
import os
import openpyxl
from cobra.flux_analysis import flux_variability_analysis
import pandas as pd
from cobra.io import load_matlab_model

# Flux Balance Analysis

In [ ]:
# make a loop to read the models and run the FBA and then store the information in xlsx format
for filename in os.listdir(input_path):
    if filename.endswith('.mat'):
        # Create the full file path
        file_path = os.path.join(input_path, filename)
        
        # Load the MATLAB model
        print(f"Loading model: {filename}")
        model = load_matlab_model(file_path)

        # Run FBA
        solution = model.optimize()
        fluxes = solution.fluxes

        # collect the information of the reaction FBA results and the other info
        flux_data = []
        for rxn in model.reactions:
            flux = fluxes[rxn.id]
            metabolite = list(rxn.metabolites.keys())[0]
            met_id = metabolite.id
            met_name = metabolite.name

            flux_data.append({
                'Reaction ID': rxn.id,
                'Reaction Name': rxn.name,
                'Flux': flux,
                'Subsystem': rxn.subsystem,
                'Metabolite ID': met_id,
                'Metabolite Name': met_name
            })  
        # Convert to DataFrames
        fluxes_df = pd.DataFrame(flux_data)

        # Save into one Excel file with three sheets: Objective, Uptake, Secretion
        excel_filename = filename.replace('.mat', '_FBA.xlsx') 
        excel_file = os.path.join(FBA_output, excel_filename)
        
        with pd.ExcelWriter(excel_file, engine="openpyxl") as writer:
            # Write Objective Value
            pd.DataFrame({'Objective Value': [solution.objective_value]}).to_excel(writer, sheet_name="Objective", index=False)

            # Write reaction fluxes
            fluxes_df.to_excel(writer, sheet_name="fluxes", index=False)

# Flux Variability Analysis

In [ ]:
# make a loop that read through each model in the file and then run FVA
# consider that I will get the FVA results while optimitmizing the solution
# reults will include the reaction ID, its associated subsystem and both mic and max fluxes
for filename in os.listdir(input_path):
    if filename.endswith('.mat'): 
        # Create the full file path
        file_path = os.path.join(input_path, filename)
        
        # Load the MATLAB model
        print(f"Loading model: {filename}")
        model = load_matlab_model(file_path)

        print(f"Running FVA for the model: {filename}")

        # Identify lipid/energy metabolism reactions
        lipid_energy_reactions = [
            rxn.id for rxn in model.reactions
            if rxn.subsystem and (
                'lipid' in rxn.subsystem.lower() or 
                'fatty acid' in rxn.subsystem.lower() or
                'cholesterol' in rxn.subsystem.lower() or
                'bile acid' in rxn.subsystem.lower() or
                'triglyceride' in rxn.subsystem.lower() or
                'glycolysis' in rxn.subsystem.lower() or
                'citric' in rxn.subsystem.lower() or
                'oxidative phosphorylation' in rxn.subsystem.lower() or
                'atp' in rxn.subsystem.lower() or
                'pentose phosphate' in rxn.subsystem.lower() or
                'pyruvate' in rxn.subsystem.lower()
            )
        ]

        # Run FVA for selected reactions
        FVA = flux_variability_analysis(model, reaction_list=lipid_energy_reactions, fraction_of_optimum=1.0)

        # Extract reaction information for only the lipid/energy reactions
        reaction_ids = []
        subsystems = []
        reaction_names = []
        genes = []

        for rxn_id in lipid_energy_reactions:
            rxn = model.reactions.get_by_id(rxn_id)
            reaction_ids.append(rxn.id)
            subsystems.append(rxn.subsystem)
            reaction_names.append(rxn.name)
            genes.append(";".join([g.id for g in rxn.genes]))

        # Create DataFrame
        df = pd.DataFrame(FVA)
        
        # Add reaction metadata
        df['reaction_id'] = reaction_ids
        df['reaction_name'] = reaction_names
        df['subsystem'] = subsystems
        df['genes'] = genes
        
        # Reorder columns
        df = df[['reaction_id', 'reaction_name', 'subsystem', 'genes', 'minimum', 'maximum']]

        # Save as CSV
        csv_filename = os.path.splitext(filename)[0] + '_FVA.csv'
        csv_file_path = os.path.join(FVA_output, csv_filename)
        df.to_csv(csv_file_path, index=False)

        print(f"Saved FVA results to {csv_file_path}")